In [1]:
"""
Basic example of AMUSE's "multiple" package.

Code developed by Steven McMillan & Simon Portegies Zwart.
Code adapted from the AMUSE textbook examples: 
https://github.com/amusecode/amuse/blob/master/examples/textbook/basic_multiples_si.py

Notebook by Francisca Concha-Ramírez.
"""

from __future__ import print_function
import numpy
from amuse.units import nbody_system, units, constants
from amuse.ic.plummer import new_plummer_model
from amuse.community.ph4.interface import ph4
from amuse.community.smalln.interface import SmallN
from amuse.community.kepler.interface import Kepler
from amuse.couple import multiples

/home/fran/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Awkward syntax here because multiples needs a function that resets
# and returns a small-N integrator.

###BOOKLISTSTART1###
SMALLN = None
def init_smalln(converter):
    global SMALLN
    SMALLN = SmallN(convert_nbody=converter)
def new_smalln():
    SMALLN.reset()
    return SMALLN
###BOOKLISTSTOP1###

def stop_smalln():
    global SMALLN
    SMALLN.stop()

In [3]:
def print_diagnostics(grav, E0=None):

    # Simple diagnostics.

    ke = grav.kinetic_energy
    pe = grav.potential_energy
    Nmul, Nbin, Emul = grav.get_total_multiple_energy()
    
    print('Time =', grav.get_time().in_(units.Myr))
    print('top-level kinetic energy =', ke)
    print('top-level potential energy =', pe)
    print('total top-level energy =', ke + pe)
    print(Nmul, 'multiples,', 'total energy =', Emul)
    
    E = ke + pe + Emul
    print('uncorrected total energy =', E)
    
    # Apply known corrections.
    
    Etid = grav.multiples_external_tidal_correction \
            + grav.multiples_internal_tidal_correction  # tidal error
    Eerr = grav.multiples_integration_energy_error  # integration error

    E -= Etid + Eerr
    print('corrected total energy =', E)

    if E0 is not None: print('relative energy error=', (E-E0)/E0)
    
    return E

In [4]:
def integrate_system(N, t_end, seed=None):

    total_mass = N | units.MSun
    length = 1 | units.parsec
    converter = nbody_system.nbody_to_si(total_mass, length)
    
    # Initialize gravity code for N-body system
    gravity = ph4(convert_nbody=converter)
    gravity.initialize_code()
    gravity.parameters.set_defaults()
    gravity.parameters.epsilon_squared = (0.0 | units.parsec) ** 2

    # Initialize stars
    if seed is not None: numpy.random.seed(seed)
    stars = new_plummer_model(N, convert_nbody=converter)
    stars.mass = total_mass / N
    stars.scale_to_standard(convert_nbody=converter,
                            smoothing_length_squared=gravity.parameters.epsilon_squared)
    
    # Set stellar ID. Important for internal bookkeeping within the module.
    id = numpy.arange(N)
    stars.id = id + 1
    stars.radius = 0.5 / N | units.parsec

    gravity.particles.add_particles(stars)

    # Set stopping condition: collision detection
    stopping_condition = gravity.stopping_conditions.collision_detection
    stopping_condition.enable()

    # Initalize smalln integrator
    init_smalln(converter)
    
    # Initialize Kepler code
    kep = Kepler(unit_converter=converter)
    kep.initialize_code()
    
    # Initialize code for multiples
    multiples_code = multiples.Multiples(gravity, new_smalln, kep, constants.G)
    multiples_code.neighbor_perturbation_limit = 0.05
    multiples_code.global_debug = 1

    # global_debug = 0: no output from multiples
    #               1: minimal output
    #               2: debugging output
    #               3: even more output

    print('multiples_code.neighbor_veto =', multiples_code.neighbor_veto)
    print('multiples_code.neighbor_perturbation_limit =', multiples_code.neighbor_perturbation_limit)
    print('multiples_code.retain_binary_apocenter =', multiples_code.retain_binary_apocenter)
    print('multiples_code.wide_perturbation_limit =', multiples_code.wide_perturbation_limit)

    time = numpy.sqrt(length**3/(constants.G*total_mass))
    print('time unit =', time.in_(units.Myr))

    E0 = print_diagnostics(multiples_code)
    multiples_code.evolve_model(t_end)
    print_diagnostics(multiples_code, E0)

    gravity.stop()
    kep.stop()
    stop_smalln()

In [5]:
N = 100
t_end = 10.0 | units.Myr
integrate_system(N, t_end, 42)

multiples_code.neighbor_veto = True
multiples_code.neighbor_perturbation_limit = 0.05
multiples_code.retain_binary_apocenter = True
multiples_code.wide_perturbation_limit = 0.01
time unit = 1.49080299942 Myr
Time = 0.0 Myr
top-level kinetic energy = 2.13908733108e+37 m**2 * kg * s**-2
top-level potential energy = -4.27817466217e+37 m**2 * kg * s**-2
total top-level energy = -2.13908733108e+37 m**2 * kg * s**-2
0 multiples, total energy = zero
uncorrected total energy = -2.13908733108e+37 m**2 * kg * s**-2
corrected total energy = -2.13908733108e+37 m**2 * kg * s**-2

Evolve model to 10.0 Myr starting at 0.0 s

interaction at time 1.64014864187e+13 s
initial top-level: 84 (1.54283879064e+14 m) 50 (1.54283879064e+14 m)
M = 3.97784e+30 kg  Etop = 6.39697180347e+35 m**2 * kg * s**-2
after encounter: 2 single(s), 0 multiple(s)
final top-level: 84 (1.54283879064e+14 m) 50 (1.54283879064e+14 m) 
M = 3.97784e+30 kg Etop = 6.39697180347e+35 m**2 * kg * s**-2

interaction at time 2.66064728915e+

/home/fran/amuse/src/amuse/units/generic_unit_converter.py:189: RuntimeWarning: overflow encountered in double_scalars
  return new_quantity(number * factor, new_unit)
/home/fran/amuse/src/amuse/units/generic_unit_converter.py:227: RuntimeWarning: overflow encountered in double_scalars
  return new_quantity(number * factor, new_unit)


M = 3.97784e+30 kg  Etop = 4.74064212517e+35 m**2 * kg * s**-2
after encounter: 2 single(s), 0 multiple(s)
final top-level: 6 (1.54283879064e+14 m) 83 (1.54283879064e+14 m) 
M = 3.97784e+30 kg Etop = 4.74064212517e+35 m**2 * kg * s**-2

interaction at time 4.03490348663e+13 s
initial top-level: 16 (1.54283879064e+14 m) 30 (1.54283879064e+14 m)
M = 3.97784e+30 kg  Etop = 4.29324257225e+35 m**2 * kg * s**-2
after encounter: 2 single(s), 0 multiple(s)
final top-level: 16 (1.54283879064e+14 m) 30 (1.54283879064e+14 m) 
M = 3.97784e+30 kg Etop = 4.29324257225e+35 m**2 * kg * s**-2

interaction at time 9.22870751922e+13 s
initial top-level: 22 (1.54283879064e+14 m) 83 (1.54283879064e+14 m)
M = 3.97784e+30 kg  Etop = 2.3307543023e+35 m**2 * kg * s**-2
after encounter: 2 single(s), 0 multiple(s)
final top-level: 22 (1.54283879064e+14 m) 83 (1.54283879064e+14 m) 
M = 3.97784e+30 kg Etop = 2.3307543023e+35 m**2 * kg * s**-2

interaction at time 9.82251483565e+13 s
initial top-level: 92 (1.542838


interaction at time 2.46642520557e+14 s
initial top-level: 41 (1.54283879064e+14 m) 33 (1.54283879064e+14 m)
encounter vetoed by 14 at distance 5.03170183601e+14 m pert = 0.226425160367

interaction at time 2.46654006192e+14 s
initial top-level: 41 (1.54283879064e+14 m) 33 (1.54283879064e+14 m)
encounter vetoed by 14 at distance 4.92860979274e+14 m pert = 0.226323746856

interaction at time 2.46665491827e+14 s
initial top-level: 41 (1.54283879064e+14 m) 33 (1.54283879064e+14 m)
encounter vetoed by 14 at distance 4.82478689296e+14 m pert = 0.227490180463

interaction at time 2.46676977462e+14 s
initial top-level: 41 (1.54283879064e+14 m) 33 (1.54283879064e+14 m)
encounter vetoed by 14 at distance 4.72024296646e+14 m pert = 0.230106698062

interaction at time 2.46688463096e+14 s
initial top-level: 41 (1.54283879064e+14 m) 33 (1.54283879064e+14 m)
encounter vetoed by 14 at distance 4.61499365636e+14 m pert = 0.234389808136

interaction at time 2.46699948731e+14 s
initial top-level: 41 (1


interaction at time 2.48241895196e+14 s
initial top-level: 41 (1.54283879064e+14 m) 14 (1.54283879064e+14 m)
M = 3.97784e+30 kg  Etop = -5.90731472186e+35 m**2 * kg * s**-2
after encounter: 0 single(s), 1 multiple(s)
splitting perturbed binary (41,14)
final top-level: 41 (1.54283879064e+14 m) 14 (1.54283879064e+14 m) 
M = 3.97784e+30 kg Etop = -7.59613978783e+36 m**2 * kg * s**-2

interaction at time 2.482433309e+14 s
initial top-level: 41 (1.54283879064e+14 m) 14 (1.54283879064e+14 m)
M = 3.97784e+30 kg  Etop = -5.90721228005e+35 m**2 * kg * s**-2
after encounter: 0 single(s), 1 multiple(s)
splitting perturbed binary (41,14)
final top-level: 41 (1.54283879064e+14 m) 14 (1.54283879064e+14 m) 
M = 3.97784e+30 kg Etop = -7.62347434838e+36 m**2 * kg * s**-2

interaction at time 2.48244766605e+14 s
initial top-level: 41 (1.54283879064e+14 m) 14 (1.54283879064e+14 m)
M = 3.97784e+30 kg  Etop = -5.90711047568e+35 m**2 * kg * s**-2
after encounter: 0 single(s), 1 multiple(s)
splitting pertur


interaction at time 3.10255708604e+14 s
initial top-level: 40 (1.54283879064e+14 m) 48 (1.54283879064e+14 m)
encounter vetoed by 50 at distance 3.96330987006e+14 m pert = 0.450159516033

interaction at time 3.10261451421e+14 s
initial top-level: 48 (1.54283879064e+14 m) 40 (1.54283879064e+14 m)
encounter vetoed by 50 at distance 3.98194524186e+14 m pert = 0.400217843644

interaction at time 3.10267194238e+14 s
initial top-level: 48 (1.54283879064e+14 m) 40 (1.54283879064e+14 m)
encounter vetoed by 50 at distance 3.99999413338e+14 m pert = 0.354486413623

interaction at time 3.10272937056e+14 s
initial top-level: 48 (1.54283879064e+14 m) 40 (1.54283879064e+14 m)
encounter vetoed by 50 at distance 4.01745186828e+14 m pert = 0.312702463104

interaction at time 3.10278679873e+14 s
initial top-level: 48 (1.54283879064e+14 m) 40 (1.54283879064e+14 m)
encounter vetoed by 50 at distance 4.03431382606e+14 m pert = 0.274619410209

interaction at time 3.10284422691e+14 s
initial top-level: 48 (1